<a href="https://colab.research.google.com/github/KarinaSelingardi/ESTUDOS-DE-PYTHON/blob/main/C%C3%B3pia_de_RandomForest_VendaCarros_Apos90dias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Floresta Randômica (*Random Forest*) 🌳🌳🌳🐍

📚🌳🌳🌳 Florestas aleatórias ou florestas de decisão aleatória é um método de aprendizado conjunto para classificação, regressão e outras tarefas que opera construindo uma infinidade de árvores de decisão em tempo de treinamento.

## Exemplo: Vendas de uma concessionária de veículos usados

Pegamos o mesmo exemplo que utilizamos em [Árvore de Decisão](https://colab.research.google.com/drive/1DjjLYneLw7pZxuKNnxFCmSgwgBsauQOS?usp=drive_fs#scrollTo=SKIUf2z6upb6&forceEdit=true&sandboxMode=true), para mostrar a melhoria em desempenho... 🏃

In [ ]:
# Necessário para ler o .xls
!pip install --upgrade xlrd

     |████████████████████████████████| 96 kB 2.9 MB/s 
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0


In [ ]:
# Importando o pandas
import pandas as pd

In [ ]:
# arquivo disponível em: https://github.com/aasouzaconsult/DiversosMachineLearning/blob/master/Data/carros_usados.xls
df = pd.read_excel('/content/carros_usados.xls')

## Analisando os dados...
- `data.set` (informa se é dado de treinamento ou teste)
- `total.cost` (custo total do veículo)
- `lot.sales.days` (dias na concessionária até a venda)
- `overage` (passou de 90 dias na concessionária?)
- `mileage` (kilometragem do veículo)
- `vehicle.type` (Tipo do veículo)
- `domestic.import` (Se é produzido no país ou importado)
- `vehicle.age` (idade do veículo)
- `vehicle.age.group` (grupo com idade do veículo)
- `color.set` (cor do veículo)
- `makex` (fabricante do veículo)
- `state` (estado (EUA) do registro veículo)
- `make.model` (modelo do veículo)

In [ ]:
# Visualizando as 5 primeiras linhas do Dataframe
df.head()

,data.set,total.cost,lot.sale.days,overage,mileage,vehicle.type,domestic.import,vehicle.age,vehicle.age.group,color.set,makex,state,make.model
0,TRAIN,4037,135,YES,67341,FAMILY.LARGE,Domestic,8,SEVEN+,SILVER,BUICK,CA,BUICK.CENTURY
1,TRAIN,6886,114,YES,72823,FAMILY.MEDIUM,Domestic,6,SIX,WHITE,DODGE,AZ,DODGE.NEON
2,TRAIN,3924,205,YES,96330,FAMILY.MEDIUM,Domestic,9,SEVEN+,WHITE,OLDSMOBILE,FL,OLDSMOBILE.CUTLASS SUPREME
3,TRAIN,4636,100,YES,58519,ECONOMY,Import,4,FOUR,GOLD,KIA,NV,KIA.SEPHIA
4,TRAIN,5080,129,YES,99057,FAMILY.MEDIUM,Domestic,4,FOUR,GREEN,OLDSMOBILE,NV,OLDSMOBILE.CUTLASS SUPREME


In [ ]:
# Visualizando as informações de colunas do Dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7495 entries, 0 to 7494
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   data.set           7495 non-null   object
 1   total.cost         7495 non-null   int64 
 2   lot.sale.days      7495 non-null   int64 
 3   overage            7495 non-null   object
 4   mileage            7495 non-null   int64 
 5   vehicle.type       7495 non-null   object
 6   domestic.import    7495 non-null   object
 7   vehicle.age        7495 non-null   int64 
 8   vehicle.age.group  7495 non-null   object
 9   color.set          7495 non-null   object
 10  makex              7495 non-null   object
 11  state              7495 non-null   object
 12  make.model         7495 non-null   object
dtypes: int64(4), object(9)
memory usage: 761.3+ KB


In [ ]:
# Informações estatísticas das colunas Dataframe
df.describe()

,total.cost,lot.sale.days,mileage,vehicle.age
count,7495.000000,7495.000000,7495.000000,7495.000000
mean,4980.035490,76.454303,77234.533422,5.567312
std,937.602064,64.644337,17142.552588,1.756219
min,1375.000000,0.000000,1036.000000,1.000000
25%,4354.000000,14.000000,67946.000000,4.000000
50%,4919.000000,73.000000,78534.000000,5.000000
75%,5540.000000,120.000000,89340.000000,7.000000
max,9813.000000,300.000000,119250.000000,12.000000


In [ ]:
# Consultando dados nulos ou Missing Values no Dataframe
df.isnull().sum()

data.set             0
total.cost           0
lot.sale.days        0
overage              0
mileage              0
vehicle.type         0
domestic.import      0
vehicle.age          0
vehicle.age.group    0
color.set            0
makex                0
state                0
make.model           0
dtype: int64

## Construindo um Modelo de Machine Learning 🤖

Nosso objetivo é prever se o veículo irá passar mais de 90 dias na concessionária sem ser vendido.

In [ ]:
# visualizando novamente os dados...
df.head()

,data.set,total.cost,lot.sale.days,overage,mileage,vehicle.type,domestic.import,vehicle.age,vehicle.age.group,color.set,makex,state,make.model
0,TRAIN,4037,135,YES,67341,FAMILY.LARGE,Domestic,8,SEVEN+,SILVER,BUICK,CA,BUICK.CENTURY
1,TRAIN,6886,114,YES,72823,FAMILY.MEDIUM,Domestic,6,SIX,WHITE,DODGE,AZ,DODGE.NEON
2,TRAIN,3924,205,YES,96330,FAMILY.MEDIUM,Domestic,9,SEVEN+,WHITE,OLDSMOBILE,FL,OLDSMOBILE.CUTLASS SUPREME
3,TRAIN,4636,100,YES,58519,ECONOMY,Import,4,FOUR,GOLD,KIA,NV,KIA.SEPHIA
4,TRAIN,5080,129,YES,99057,FAMILY.MEDIUM,Domestic,4,FOUR,GREEN,OLDSMOBILE,NV,OLDSMOBILE.CUTLASS SUPREME


**Pre-processando os dados**

- Precisamos pré-processar algumas colunas, pois, são colunas categóricas.
- Nesta etapa também removemos colunas não importantes para o modelo.

Removendo as colunas `vehicle.age.group` e `data.set` (não irão nos interessar nesta análise)

In [ ]:
df.drop('vehicle.age.group', axis=1, inplace=True)

In [ ]:
df.drop('data.set', axis=1, inplace=True)

**Aplicando o LabelEncoder para codificação de variáveis categóricas** 💡

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
label_encoder1 = LabelEncoder()
label_encoder2 = LabelEncoder()
label_encoder3 = LabelEncoder()
label_encoder4 = LabelEncoder()
label_encoder5 = LabelEncoder()
label_encoder6 = LabelEncoder()

**Aplica o Labelencoder nos dados categóricos**

In [ ]:
df['vehicle.type'] = label_encoder1.fit_transform(df['vehicle.type'])
df['domestic.import'] = label_encoder2.fit_transform(df['domestic.import'])
df['color.set'] = label_encoder3.fit_transform(df['color.set'])
df['makex'] = label_encoder4.fit_transform(df['makex'])
df['state'] = label_encoder5.fit_transform(df['state'])
df['make.model'] = label_encoder6.fit_transform(df['make.model'])

**Dados transformados** 🆗

In [ ]:
df.head()

,total.cost,lot.sale.days,overage,mileage,vehicle.type,domestic.import,vehicle.age,color.set,makex,state,make.model
0,4037,135,YES,67341,1,0,8,6,0,1,1
1,6886,114,YES,72823,2,0,6,7,5,0,33
2,3924,205,YES,96330,2,0,9,7,15,2,77
3,4636,100,YES,58519,0,1,4,2,10,5,54
4,5080,129,YES,99057,2,0,4,3,15,5,77


**Separa a classe dos dados**

In [ ]:
# nossa variável target será overage (prever se o veículo passará ou não 90 dias na concessionária)
y = df['overage']

In [ ]:
y.head()

0    YES
1    YES
2    YES
3    YES
4    YES
Name: overage, dtype: object

Apaga as colunas `overage` e `lot.days.sales` para não enviesar o modelo

In [ ]:
# variáveis preditoras (X)
X = df.drop(['lot.sale.days','overage'], axis=1)

In [ ]:
# Visualizando nosso X
X.head()

,total.cost,mileage,vehicle.type,domestic.import,vehicle.age,color.set,makex,state,make.model
0,4037,67341,1,0,8,6,0,1,1
1,6886,72823,2,0,6,7,5,0,33
2,3924,96330,2,0,9,7,15,2,77
3,4636,58519,0,1,4,2,10,5,54
4,5080,99057,2,0,4,3,15,5,77


**Separando os dados de treino e teste**

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y)

**Aplicando Machine Learning com Random Forest** 🌳🌳🌳

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# n_estimators = 100 - número de árvores na floresta (padrão é 10)
# criterion='entropy'
# random_state=0 (gerar sempre o mesmo resultado)

floresta = RandomForestClassifier(n_estimators = 100, criterion='entropy', random_state=1)
floresta.fit(X_treino, y_treino) # treinando o modelo

RandomForestClassifier(criterion='entropy', random_state=1)

**Validação do Modelo**

In [ ]:
X_teste.head()

,total.cost,mileage,vehicle.type,domestic.import,vehicle.age,color.set,makex,state,make.model
6310,6690,91710,4,1,6,4,14,6,67
5516,6413,53504,0,0,4,1,5,3,33
885,4561,101108,2,0,8,6,18,0,82
4887,4528,105202,2,1,9,4,19,0,92
5005,5691,88678,2,1,6,3,19,0,92


In [ ]:
# testando o modelo
floresta.predict(X_teste)

array(['YES', 'NO', 'NO', ..., 'NO', 'NO', 'NO'], dtype=object)

In [ ]:
resultado = floresta.predict(X_teste)

In [ ]:
resultado

array(['NO', 'YES', 'YES', ..., 'YES', 'NO', 'NO'], dtype=object)

Visualizando as métricas do nosso modelo...

In [ ]:
from sklearn import metrics
print(metrics.classification_report(y_teste,resultado))

              precision    recall  f1-score   support

          NO       0.70      0.75      0.72       986
         YES       0.70      0.64      0.67       888

    accuracy                           0.70      1874
   macro avg       0.70      0.70      0.70      1874
weighted avg       0.70      0.70      0.70      1874



Estes foram os resultado para [Árvore de Decisão](https://colab.research.google.com/drive/1DjjLYneLw7pZxuKNnxFCmSgwgBsauQOS?usp=drive_fs#scrollTo=SKIUf2z6upb6&forceEdit=true&sandboxMode=true), que vimos anteriormente...

              precision    recall  f1-score   support
          NO       0.64      0.63      0.64       989
         YES       0.60      0.61      0.60       885

    accuracy                           0.62      1874
    macro avg      0.62      0.62      0.62      1874
    weighted avg   0.62      0.62      0.62      1874


***Agora, com Random Forest, observem como o desempenho do modelo melhorou:***
- Tivemos uma precisão de 67% para classificar carros que superam os 90 dias.
- O modelo pode melhorar ainda mais se trabalharmos mais nos dados, e com um trabalho de `engenharia de features`.

Matrix de Confusão

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
previsoes = floresta.predict(X_teste)
confusao = confusion_matrix(y_teste, resultado)
confusao

array([[739, 247],
       [319, 569]])

Taxa de acerto

In [ ]:
taxa_acerto = accuracy_score(y_teste, previsoes)
taxa_acerto

0.6979722518676628

Estimadores

In [ ]:
# mostra as 100 árvores (n_estimators)
floresta.estimators_

[DecisionTreeClassifier(criterion='entropy', max_features='auto',
                        random_state=1791095845),
 DecisionTreeClassifier(criterion='entropy', max_features='auto',
                        random_state=2135392491),
 DecisionTreeClassifier(criterion='entropy', max_features='auto',
                        random_state=946286476),
 DecisionTreeClassifier(criterion='entropy', max_features='auto',
                        random_state=1857819720),
 DecisionTreeClassifier(criterion='entropy', max_features='auto',
                        random_state=491263),
 DecisionTreeClassifier(criterion='entropy', max_features='auto',
                        random_state=550290313),
 DecisionTreeClassifier(criterion='entropy', max_features='auto',
                        random_state=1298508491),
 DecisionTreeClassifier(criterion='entropy', max_features='auto',
                        random_state=2143362693),
 DecisionTreeClassifier(criterion='entropy', max_features='auto',
            

## Mais informações sobre Machine Learning?

Confira no Blog do Alex Souza ([Medium](https://medium.com/blog-do-zouza/o-que-%C3%A9-machine-learning-5e7e98453985))

Mais dicas diárias, confira nosso [Instagram](https://www.instagram.com/alexsouzamsc/).